# Clustering Documentation Example

<h2 id="k-means">K-means</h2>

<p><a href="http://en.wikipedia.org/wiki/K-means_clustering">k-means</a> is one of the
most commonly used clustering algorithms that clusters the data points into a
predefined number of clusters. The MLlib implementation includes a parallelized
variant of the <a href="http://en.wikipedia.org/wiki/K-means%2B%2B">k-means++</a> method
called <a href="http://theory.stanford.edu/~sergei/papers/vldb12-kmpar.pdf">kmeans||</a>.</p>

<p><code>KMeans</code> is implemented as an <code>Estimator</code> and generates a <code>KMeansModel</code> as the base model.</p>

<h3 id="input-columns">Input Columns</h3>

<table class="table">
  <thead>
    <tr>
      <th align="left">Param name</th>
      <th align="left">Type(s)</th>
      <th align="left">Default</th>
      <th align="left">Description</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>featuresCol</td>
      <td>Vector</td>
      <td>"features"</td>
      <td>Feature vector</td>
    </tr>
  </tbody>
</table>

<h3 id="output-columns">Output Columns</h3>

<table class="table">
  <thead>
    <tr>
      <th align="left">Param name</th>
      <th align="left">Type(s)</th>
      <th align="left">Default</th>
      <th align="left">Description</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>predictionCol</td>
      <td>Int</td>
      <td>"prediction"</td>
      <td>Predicted cluster center</td>
    </tr>
  </tbody>
</table>

In [2]:
#Cluster methods Example
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('cluster').getOrCreate()

In [3]:
from pyspark.ml.clustering import KMeans

# Loads data.
dataset = spark.read.format("libsvm").load("/FileStore/tables/sample_kmeans_data.txt")

dataset.show()

+-----+--------------------+
label| features|
+-----+--------------------+
 0.0| (3,[],[])|
 1.0|(3,[0,1,2],[0.1,0...|
 2.0|(3,[0,1,2],[0.2,0...|
 3.0|(3,[0,1,2],[9.0,9...|
 4.0|(3,[0,1,2],[9.1,9...|
 5.0|(3,[0,1,2],[9.2,9...|
+-----+--------------------+

In [4]:
# Trains a k-means model.
kmeans = KMeans(featuresCol='features').setK(2).setSeed(1)
model = kmeans.fit(dataset)

# Evaluate clustering by computing Within Set Sum of Squared Errors.
wssse = model.summary.trainingCost
print("Within Set Sum of Squared Errors = " + str(wssse))

# Shows the result.
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

Within Set Sum of Squared Errors = 0.11999999999994547
Cluster Centers: 
[9.1 9.1 9.1]
[0.1 0.1 0.1]

In [5]:
# Prediction value is the K generated for the row register values

results = model.transform(dataset)

results.show()

+-----+--------------------+----------+
label| features|prediction|
+-----+--------------------+----------+
 0.0| (3,[],[])| 1|
 1.0|(3,[0,1,2],[0.1,0...| 1|
 2.0|(3,[0,1,2],[0.2,0...| 1|
 3.0|(3,[0,1,2],[9.0,9...| 0|
 4.0|(3,[0,1,2],[9.1,9...| 0|
 5.0|(3,[0,1,2],[9.2,9...| 0|
+-----+--------------------+----------+

Alright let's code through our own example!